In [ ]:
#import all necessary libaries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install lxml --yes
import lxml

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::_ipyw_jlab_nb_ext_conf==0.1.0=py37_0
  - defaults/linux-64::alabaster==0.7.11=py37_0
  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::anaconda-client==1.7.2=py37_0
  - defaults/linux-64::anaconda-navigator==1.9.2=py37_0
  - defaults/linux-64::anaconda-project==0.8.2=py37_0
  - defaults/linux-64::appdirs==1.4.3=py37h28b3542_0
  - defaults/linux-64::asn1crypto==0.24.0=py37_0
  - defaults/linux-64::astroid==2.0.4=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::atomicwrites==1.2.1=py37_0
  - defaults/linux-64::automat==0.7.0=py37_0
  - defaults/linux-64::babel==2.6.0=py37_0
  - defaults/linux-64::backports==1.0=py37_1
  - defaults/linux-64::backports.shutil_get_terminal_size==1.0.

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [3]:
#drop Not assigned Boroughs and Neighbourhoods
df=df[df['Borough']!='Not assigned']
df=df[df['Neighbourhood']!='Not assigned']

In [4]:
#Group the data by postcode and join the neighbourhoods in one row
df=df.groupby('Postcode',as_index=False).agg({'Borough' : 'first','Neighbourhood' : ', '.join })

In [5]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
df.shape

(102, 3)

In [19]:
#import coordinates data
path = 'http://cocl.us/Geospatial_data'
coords = pd.read_csv(path) 

In [20]:
coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [25]:
#merge coordinates data to dataframe by postcode
toronto_data = pd.merge(left=df,right=coords, how='left', left_on='Postcode', right_on='Postal Code')
toronto_data.drop('Postal Code',axis=1,inplace=True)

In [26]:
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
97,M9N,York,Weston,43.706876,-79.518188
98,M9P,Etobicoke,Westmount,43.696319,-79.532242
99,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
100,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
